In [ ]:
from google.colab import drive
from timeit import default_timer as timer
drive.mount('/content/drive')

Mounted at /content/drive


### Einlesen der Instanzen



In [ ]:
from sre_constants import ANY
def read(filename):
    with open(filename) as f:
        line = f.readline().rstrip().split()
        n = int(line[0])
        line = f.readline().rstrip().split()
        node_weights = [int(i) for i in line]
        edge_weights = {}
        for i in range(n):
            line = [int(w) for w in f.readline().rstrip().split()]
            for j, w in enumerate(line):
                if j >= 0 and w >= 0:
                    edge_weights[i, j] = w
        return n, node_weights, edge_weights

/tmp/ipython-input-2-941494749.py:1: DeprecationWarning: module 'sre_constants' is deprecated
  from sre_constants import ANY


### Graph erstellen

In [ ]:
import networkx as nx
def erstelle_graph(instance):
  n, node_weights, edge_weights = instance
  G = nx.Graph()

  for i in range(n):
    G.add_node(i,weight = node_weights[i])

  for (i, j), w in edge_weights.items():
    if w > 0:
      G.add_edge(i, j)

  return G

### Greedy(Rollout)


In [ ]:
def select_using_rollout(G, cleared, current, instance):
  sim_cleared = set(cleared)  #set enthät die bereits besuchten knoten
  simulierter_path = [current]  #start der Liste für den simulierten pfad
  sim_cleared.add(current)  #current wird als startknoten dem set hunzugefügt

  while len(sim_cleared) < G.number_of_nodes(): #schleife läuft solange es noch unbesuchte räume gibt
    candidates = [n for n in G.neighbors(current) if n not in sim_cleared]  #liste der unbesuchten nachbarn des aktuellen raumes
    if not candidates:
      candidates = [i for i in G.nodes() if i not in sim_cleared] #liste aller unbesuchten räume
    next_node = min(candidates, key=lambda x: len([n for n in G.neighbors(x) if n not in sim_cleared])) #greedy kriterium
    simulierter_path.append(next_node)
    sim_cleared.add(next_node)
    current = next_node
  return simulierter_path #gebe den simulierten Pfad zurück

def greedy_graph_rollout(instance, start = 0):
  G = erstelle_graph(instance)
  n, node_weights, edge_weights = instance

  cleared = set() #set enthät die bereits besuchten räume
  path = [start]  #liste speichert die reihenfolge
  current = start #startknoten wird unser aktuell besuchter raum
  cleared.add(current)  #startknoten wird in unser set hinzugefügt

  while len(cleared) < G.number_of_nodes(): #schleife läuft bis alle räume besucht wurden
    candidates = [n for n in G.neighbors(current) if n not in cleared]  #liste der unbesuchten nachbarn
    if not candidates:
      candidates =  [i for i in range(n) if i not in cleared] #liste aller unbesuchten räume
    best_candidate = None  #speicher des besten kandidaten
    best_sim_cost = 1000000#speicher der besten geschätzten kosten

    for i in candidates:
      pre_cleared = cleared.copy()
      pre_cleared.add(i)
      sim_path = select_using_rollout(G, pre_cleared, i, instance)

      sim_cost,_ = Graph_clearer(instance, sim_path)
      if sim_cost < best_sim_cost:
        best_sim_cost = sim_cost
        best_candidate = i

    path.append(best_candidate)
    cleared.add(best_candidate)
    current = best_candidate
  return path

### Greedy(wenigste nächsten Knoten)


In [ ]:

import networkx as nx


def greedy_graph_clear_min_neighbors(instance, start=0):
    n, node_weights, edge_weights = instance
    G = nx.DiGraph()  #erstellt graphne aus der instanz
    for i in range(n):
      G.add_node(i,weight = node_weights[i])

    for (i, j), w in edge_weights.items():
        if w > 0:
            G.add_edge(i, j)

    cleared = set() #besuchte knoten
    path = [start]  #reihenfolge
    current = start #knoten den wir gerade betrachten
    cleared.add(current)  #aktuell besuchter knoten wird zu der menge der besuchten knoten hinzugefuegt

    while len(cleared) < n: #schleife, die solange geht bis alle knoten besucht wurden
        candidates = [nbr for nbr in G.successors(current) if nbr not in cleared] #suche nach nachbarn die noch nicht besucht wurden
        if not candidates:  #falls es keine mehr gibt, suche nach einem neuen knoten der noch nicht besucht wurde und moeglichst wenige angrenzende knoten hat
            candidates = [i for i in range(n) if i not in cleared]
        #knoten mit den wenigstens angrenzendend knoten
        next_node = min(candidates, key=lambda x: len([pred for pred in G.predecessors(x) if pred not in cleared])) #greedy kriterium, knoten die moeglichst wenige nachbarn haben

        path.append(next_node)
        cleared.add(next_node)
        current = next_node

    return path


CPU times: user 7 µs, sys: 1 µs, total: 8 µs
Wall time: 10.5 µs


### Simplified Rollout

In [ ]:
def simplified_select_using_rollout_min_n(G, cleared, current, instance):
  sim_cleared = set(cleared)
  simulierter_path = [current]
  sim_cleared.add(current)

  while len(sim_cleared) < G.number_of_nodes():
    candidates = [n for n in G.neighbors(current) if n not in sim_cleared]
    if not candidates:
      candidates = [i for i in G.nodes() if i not in sim_cleared]

    sorted_candidates = sorted(candidates, key = lambda x: len([n for n in G.neighbors(x) if n not in sim_cleared]))

    best_node = sorted_candidates[0]

    simulierter_path.append(best_node)
    sim_cleared.add(best_node)
    current = best_node
  return simulierter_path

def simplified_select_using_rollout_node_w(G, cleared, current, instance):
  n, node_weights, edge_weights = instance
  sim_cleared = set(cleared)
  simulierter_path = [current]
  sim_cleared.add(current)

  while len(sim_cleared) < G.number_of_nodes():
    candidates = [n for n in G.neighbors(current) if n not in sim_cleared]
    if not candidates:
      candidates = [i for i in G.nodes() if i not in sim_cleared]

    sorted_candidates = sorted(candidates, key=lambda x: node_weights[x]+edge_weights[current,x])

    best_node = sorted_candidates[0]

    simulierter_path.append(best_node)
    sim_cleared.add(best_node)
    current = best_node
  return simulierter_path



def simplified_greedy_graph_rollout_min_n(instance,start=0, max_neighbors=3):
  G = erstelle_graph(instance)
  n, node_weights, edge_weights = instance

  cleared = set()
  path = [start]
  current = start
  cleared.add(current)

  while len(cleared) < G.number_of_nodes():
    candidates = [n for n in G.neighbors(current) if n not in cleared]
    if not candidates:
      candidates = [i for i in range(n) if i not in cleared]

    best_candidate = None
    best_sim_cost = 1000000

    sorted_candidates = sorted(candidates, key = lambda x: len([n for n in G.neighbors(x) if n not in cleared]))
    limited_candidates = sorted_candidates[:max_neighbors]

    for i in limited_candidates:
      pre_cleared = cleared.copy()
      pre_cleared.add(i)
      sim_path = simplified_select_using_rollout_min_n(G, pre_cleared, i, instance)

      sim_cost,_ = Graph_clearer(instance, sim_path)
      if sim_cost < best_sim_cost:
        best_sim_cost = sim_cost
        best_candidate = i

    path.append(best_candidate)
    cleared.add(best_candidate)
    current = best_candidate
  return path




def simplified_greedy_graph_rollout_node_w(instance,start=0, max_neighbors=3):
  G = erstelle_graph(instance)
  n, node_weights, edge_weights = instance

  cleared = set()
  path = [start]
  current = start
  cleared.add(current)

  while len(cleared) < G.number_of_nodes():
    candidates = [n for n in G.neighbors(current) if n not in cleared]
    if not candidates:
      candidates = [i for i in range(n) if i not in cleared]

    best_candidate = None
    best_sim_cost = 1000000

    sorted_candidates = sorted(candidates, key = lambda x: len([n for n in G.neighbors(x) if n not in cleared]))
    limited_candidates = sorted_candidates[:max_neighbors]

    for i in limited_candidates:
      pre_cleared = cleared.copy()
      pre_cleared.add(i)
      sim_path = simplified_select_using_rollout_node_w(G, pre_cleared, i, instance)

      sim_cost,_ = Graph_clearer(instance, sim_path)
      if sim_cost < best_sim_cost:
        best_sim_cost = sim_cost
        best_candidate = i

    path.append(best_candidate)
    cleared.add(best_candidate)
    current = best_candidate
  return path

### Greedy(Node Weights + Edge weights)


In [ ]:

import networkx as nx
def greedy_graph_clear(instance, start=0):
    n, node_weights, edge_weights = instance
    G = nx.DiGraph()  #erstellt graphne aus der instanz

    for i in range(n):
      G.add_node(i,weight = node_weights[i])

    for (i, j), w in edge_weights.items():
        if w > 0:
            G.add_edge(i, j)

    cleared = set() #besuchte knoten
    path = [start]  #reihenfolge
    current = start #knoten den wir gerade betrachten
    cleared.add(current)  #aktuell besuchter knoten wird zu der menge der besuchten knoten hinzugefuegt

    while len(cleared) < n: #schleife, die solange geht bis alle knoten besucht wurden
        candidates = [nbr for nbr in G.successors(current) if nbr not in cleared] #suche nach nachbarn die noch nicht besucht wurden
        if not candidates:  #falls es keine mehr gibt, suche nach einem neuen knoten der noch nicht besucht wurde und moeglichst wenige angrenzende knoten hat
            candidates = [i for i in range(n) if i not in cleared]
        #knoten mit den wenigstens angrenzendend knoten
        next_node = min(candidates, key=lambda x: node_weights[x]+edge_weights[current,x]) #greedy kriterium, knoten die moeglichst wenige nachbarn haben

        path.append(next_node)
        cleared.add(next_node)
        current = next_node

    return path


CPU times: user 13 µs, sys: 0 ns, total: 13 µs
Wall time: 16 µs


### Graph Clearer


In [ ]:
%%time
def Graph_clearer(Instanz,Weg):
  _,Räume,Durchgänge = Instanz
  Roboter_gesamt = 0
  Aktuelle_Roboter = 0
  Bereits_gereinigt = set()
  Durchgänge_bewacht = set()

  for node in Weg:
    Aktuelle_Roboter += Räume[node]
    Bereits_gereinigt.add(node)
    for i in range(len(Räume)):
      if i not in Bereits_gereinigt:
        kante = (node,i)
        kante_r = (i,node)

        if kante_r not in Durchgänge_bewacht and Durchgänge[kante] > 0:
          Aktuelle_Roboter += Durchgänge[kante]
          Durchgänge_bewacht.add(kante)
    if Aktuelle_Roboter > Roboter_gesamt:
      Roboter_gesamt += Aktuelle_Roboter - Roboter_gesamt
      Aktuelle_Roboter -= Räume[node]
      for n in range(len(Räume)):
        if node and n in Bereits_gereinigt and Durchgänge[node,n] > 0:
          Aktuelle_Roboter -= Durchgänge[(n,node)]
    else:
      for n in range(len(Räume)):
        if node and n in Bereits_gereinigt:
          Aktuelle_Roboter -= Durchgänge[(node,n)]
      Aktuelle_Roboter -= Räume[node]
  return Roboter_gesamt,Weg


CPU times: user 4 µs, sys: 0 ns, total: 4 µs
Wall time: 6.91 µs


### Multi Start


In [ ]:
def Graph_multi(filename):
  Instanz = read(filename)
  _,Räume,Durch = Instanz
  beste_roboter = 100000
  bester_Weg = []
  for n in range(len(Räume)):
    Robot,Weg = Gc_min_Neighbors(filename,n)
    if Robot < beste_roboter:
      beste_roboter = Robot
      bester_Weg = Weg
  return beste_roboter,bester_Weg

In [ ]:
def Graph_multi_node_w(filename):
  Instanz = read(filename)
  _,Räume,Durch = Instanz
  beste_roboter = 100000
  bester_Weg = []
  for n in range(len(Räume)):
    Robot,Weg = Gc_node_w(filename,n)
    if Robot < beste_roboter:
      beste_roboter = Robot
      bester_Weg = Weg
  return beste_roboter,bester_Weg

In [ ]:
def Graph_multi_rollout(filename):
  instanz = read(filename)
  _,Räume,Durch = instanz
  beste_roboter = 100000
  bester_Weg = []
  for n in range(len(Räume)):
    Weg = greedy_graph_rollout(instanz,n)
    Robot,_ = Graph_clearer(instanz,Weg)
    if Robot < beste_roboter:
      beste_roboter = Robot
      bester_Weg = Weg
  return beste_roboter,bester_Weg

In [ ]:
def Gm_simplified_min_n(filename,max_neighbors = 3):
  instanz = read(filename)
  _,Räume,Durch = instanz
  beste_roboter = 100000
  bester_Weg = []
  for n in range(len(Räume)):
    Weg = simplified_greedy_graph_rollout_min_n(instanz,n,max_neighbors)
    Robot,_ = Graph_clearer(instanz,Weg)
    if Robot < beste_roboter:
      beste_roboter = Robot
      bester_Weg = Weg
  return beste_roboter,bester_Weg

In [ ]:
def Gm_simplified_node_w(filename,max_neighbors = 3):
  instanz = read(filename)
  _,Räume,Durch = instanz
  beste_roboter = 100000
  bester_Weg = []
  for n in range(len(Räume)):
    Weg = simplified_greedy_graph_rollout_node_w(instanz,n,max_neighbors)
    Robot,_ = Graph_clearer(instanz,Weg)
    if Robot < beste_roboter:
      beste_roboter = Robot
      bester_Weg = Weg
  return beste_roboter,bester_Weg

### Proof Feasible

In [ ]:
def validate(n, node_weights, edge_weights, solution, cost):
    actual_cost = 0
    clean = set()

    for i in solution:
        if i < 0 or i > n - 1:
            print("Node {} does not exist".format(i))
            return False
        if i in clean:
            print("{} is already clean".format(i))
            return False

        n_robots = node_weights[i]
        for j in range(n):
            if (i, j) in edge_weights:
                n_robots += edge_weights[i, j]
            elif (j, i) in edge_weights:
                n_robots += edge_weights[j, i]
        for j in range(n):
            if j in clean:
                for k in range(n):
                    if k != i and k not in clean:
                        if (j, k) in edge_weights:
                            n_robots += edge_weights[j, k]
                        elif (k, j) in edge_weights:
                            n_robots += edge_weights[k, j]

        actual_cost = max(actual_cost, n_robots)
        clean.add(i)

    if len(clean) != n:
        print("The number of swept nodes is {}, but should be {}".format(len(clean), n))
        return False

    if actual_cost != cost:
        print(
            "The cost of solution {} mismatches the actual cost {}".format(
                cost, actual_cost
            )
        )
        return False

    return True

In [ ]:
def proof_feasible(filename,Anzahl_Roboter,Durchlauf):
  Instanz = read(filename)
  _,Räume,Durch = Instanz
  big = 0
  Zahl = 0
  b = 0
  if len(Räume) == len(Durchlauf):    #Länge der Räume und des abgegangen Weges muss gleich sein
    for i in range(len(Räume)):
      Zahl = Räume[i]
      for n in range(len(Räume)):           #Mindestanzahl an gebrauchten Roboter, muss die berechneten Roboter unterschreiten
        Zahl += Durch[i,n]
      if Zahl > big:
        big = Zahl
    if validate(len(Räume),Räume,Durch,Durchlauf,Anzahl_Roboter):
      b = 10
    if big <= Anzahl_Roboter and b == 10:
      print(f"Antwort gültig, Anzahl an Robotern: {Anzahl_Roboter}")
    else:
      print("Ungültig")



### Optimum Rechner


In [ ]:
def opt_rechner(filename):
  Instanz = read(filename)
  _,Räume,Durch = Instanz

  big = 0
  Zahl = 0
  for i in range(len(Räume)):         #Kanten und Knotengewichte werden summiert, für die Mindestanhzahl und vermutlichem Optimum
      Zahl = Räume[i]
      for n in range(len(Räume)):
        Zahl += Durch[i,n]
      if Zahl > big:
        big = Zahl
  return big



### Graph Clearer mit Greedy

In [ ]:
def Gc_min_Neighbors(filename,start = 0):
  Instanz = read(filename)
  Weg = greedy_graph_clear_min_neighbors(Instanz,start)
  _,Räume,Durchgänge = Instanz
  Roboter_gesamt = 0
  Aktuelle_Roboter = 0
  Bereits_gereinigt = set()
  Durchgänge_bewacht = set()

  for node in Weg:
    Aktuelle_Roboter += Räume[node]  # Knotengewicht wird addiert
    Bereits_gereinigt.add(node)
    for i in range(len(Räume)):
      if i not in Bereits_gereinigt:
        kante = (node,i)              # Kanten werden abgespeichert
        kante_r = (i,node)

        if kante_r not in Durchgänge_bewacht and Durchgänge[kante] > 0: #Kanten werden geprüft
          Aktuelle_Roboter += Durchgänge[kante]
          Durchgänge_bewacht.add(kante)
    if Aktuelle_Roboter > Roboter_gesamt:           # Neue Anzahl Roboter werden gespeichert
      Roboter_gesamt += Aktuelle_Roboter - Roboter_gesamt
      Aktuelle_Roboter -= Räume[node]
      for n in range(len(Räume)):
        if node and n in Bereits_gereinigt and Durchgänge[node,n] > 0:
          Aktuelle_Roboter -= Durchgänge[(n,node)]
    else:
      for n in range(len(Räume)):
        if node and n in Bereits_gereinigt:       # Aktuelle Roboter werden berechnet falls die neuen nicht größer sind als die Alten
          Aktuelle_Roboter -= Durchgänge[(node,n)]
      Aktuelle_Roboter -= Räume[node]
  return Roboter_gesamt,Weg

In [ ]:
def Gc_node_w(filename,start = 0):
  Instanz = read(filename)
  _,Räume,Durchgänge = Instanz
  Weg = greedy_graph_clear(Instanz,start)
  Roboter_gesamt = 0
  Aktuelle_Roboter = 0
  Bereits_gereinigt = set()
  Durchgänge_bewacht = set()

  for node in Weg:
    Aktuelle_Roboter += Räume[node]
    Bereits_gereinigt.add(node)
    for i in range(len(Räume)):
      if i not in Bereits_gereinigt:
        kante = (node,i)
        kante_r = (i,node)

        if kante_r not in Durchgänge_bewacht and Durchgänge[kante] > 0:
          Aktuelle_Roboter += Durchgänge[kante]
          Durchgänge_bewacht.add(kante)
    if Aktuelle_Roboter > Roboter_gesamt:
      Roboter_gesamt += Aktuelle_Roboter - Roboter_gesamt
      Aktuelle_Roboter -= Räume[node]
      for n in range(len(Räume)):
        if node and n in Bereits_gereinigt and Durchgänge[node,n] > 0:
          Aktuelle_Roboter -= Durchgänge[(n,node)]
    else:
      for n in range(len(Räume)):
        if node and n in Bereits_gereinigt:
          Aktuelle_Roboter -= Durchgänge[(node,n)]
      Aktuelle_Roboter -= Räume[node]
  return Roboter_gesamt,Weg

In [ ]:
def Gc_rollout(filename,start = 0):
  Instanz = read(filename)
  _,Räume,Durchgänge = Instanz
  Weg = greedy_graph_rollout(Instanz,start)
  Roboter_gesamt = 0
  Aktuelle_Roboter = 0
  Bereits_gereinigt = set()
  Durchgänge_bewacht = set()

  for node in Weg:
    Aktuelle_Roboter += Räume[node]
    Bereits_gereinigt.add(node)
    for i in range(len(Räume)):
      if i not in Bereits_gereinigt:
        kante = (node,i)
        kante_r = (i,node)

        if kante_r not in Durchgänge_bewacht and Durchgänge[kante] > 0:
          Aktuelle_Roboter += Durchgänge[kante]
          Durchgänge_bewacht.add(kante)
    if Aktuelle_Roboter > Roboter_gesamt:
      Roboter_gesamt += Aktuelle_Roboter - Roboter_gesamt
      Aktuelle_Roboter -= Räume[node]
      for n in range(len(Räume)):
        if node and n in Bereits_gereinigt and Durchgänge[node,n] > 0:
          Aktuelle_Roboter -= Durchgänge[(n,node)]
    else:
      for n in range(len(Räume)):
        if node and n in Bereits_gereinigt:
          Aktuelle_Roboter -= Durchgänge[(node,n)]
      Aktuelle_Roboter -= Räume[node]
  return Roboter_gesamt,Weg

In [ ]:
def Gc_simplified_min_n(filename,start = 0,max_numbers_rollout = 3):
  Instanz = read(filename)
  _,Räume,Durchgänge = Instanz
  Weg = simplified_greedy_graph_rollout_min_n(Instanz,start,max_numbers_rollout)
  Roboter_gesamt = 0
  Aktuelle_Roboter = 0
  Bereits_gereinigt = set()
  Durchgänge_bewacht = set()

  for node in Weg:
    Aktuelle_Roboter += Räume[node]
    Bereits_gereinigt.add(node)
    for i in range(len(Räume)):
      if i not in Bereits_gereinigt:
        kante = (node,i)
        kante_r = (i,node)

        if kante_r not in Durchgänge_bewacht and Durchgänge[kante] > 0:
          Aktuelle_Roboter += Durchgänge[kante]
          Durchgänge_bewacht.add(kante)
    if Aktuelle_Roboter > Roboter_gesamt:
      Roboter_gesamt += Aktuelle_Roboter - Roboter_gesamt
      Aktuelle_Roboter -= Räume[node]
      for n in range(len(Räume)):
        if node and n in Bereits_gereinigt and Durchgänge[node,n] > 0:
          Aktuelle_Roboter -= Durchgänge[(n,node)]
    else:
      for n in range(len(Räume)):
        if node and n in Bereits_gereinigt:
          Aktuelle_Roboter -= Durchgänge[(node,n)]
      Aktuelle_Roboter -= Räume[node]
  return Roboter_gesamt,Weg

In [ ]:
def Gc_simplified_node_w(filename,start = 0,max_numbers_rollout = 3):
  Instanz = read(filename)
  _,Räume,Durchgänge = Instanz
  Weg = simplified_greedy_graph_rollout_node_w(Instanz,start,max_numbers_rollout)
  Roboter_gesamt = 0
  Aktuelle_Roboter = 0
  Bereits_gereinigt = set()
  Durchgänge_bewacht = set()

  for node in Weg:
    Aktuelle_Roboter += Räume[node]
    Bereits_gereinigt.add(node)
    for i in range(len(Räume)):
      if i not in Bereits_gereinigt:
        kante = (node,i)
        kante_r = (i,node)

        if kante_r not in Durchgänge_bewacht and Durchgänge[kante] > 0:
          Aktuelle_Roboter += Durchgänge[kante]
          Durchgänge_bewacht.add(kante)
    if Aktuelle_Roboter > Roboter_gesamt:
      Roboter_gesamt += Aktuelle_Roboter - Roboter_gesamt
      Aktuelle_Roboter -= Räume[node]
      for n in range(len(Räume)):
        if node and n in Bereits_gereinigt and Durchgänge[node,n] > 0:
          Aktuelle_Roboter -= Durchgänge[(n,node)]
    else:
      for n in range(len(Räume)):
        if node and n in Bereits_gereinigt:
          Aktuelle_Roboter -= Durchgänge[(node,n)]
      Aktuelle_Roboter -= Räume[node]
  return Roboter_gesamt,Weg

### Ordner durchgehen

In [ ]:
import os
def verarbeite_ordner(pfad_zum_ordner):
  Files = []
  for dateiname in os.listdir(pfad_zum_ordner):
      dateipfad = os.path.join(pfad_zum_ordner, dateiname)
      if os.path.isfile(dateipfad):
        Files.append(dateipfad)
  return Files


ordner_klein = '/content/drive/MyDrive/Instanzen_klein'  # 5 20er, 30er,40er
ordner_groß = '/content/drive/MyDrive/Instanzen_groß'    # 2 100er


### Algotithmen durchführen (Data Frame)

In [ ]:

def run_algorithm_with_parameters(instance_name, function_algorithm, algorithm_parameters):

    # Parameter dictornary
    name = os.path.basename(instance_name)
    result_dict = {}
    result_dict["Instance"] = name
    result_dict["Algorithm"] = function_algorithm.__name__
    result_dict["Parameters"] = algorithm_parameters.copy()


    # ausführen
    starttime = timer()
    permutation, distance = function_algorithm(instance_name, **algorithm_parameters)


    result_dict["Objective"] = permutation
    result_dict["Optimum"] = opt_rechner(instance_name)
    result_dict["Time"] = timer()- starttime

    return result_dict

In [ ]:
def run_algorithms_with_parameters(instance_name, algorithms_parameters):

    results = []
    for algorithm, parameters in algorithms_parameters:

        results.append (run_algorithm_with_parameters(instance_name, algorithm, parameters) )



    return results


def run_algorithms_with_parameters_on_instances (instance_names, algorithms_parameters):

    results = []

    for instance_name in instance_names:
        results += run_algorithms_with_parameters( instance_name, algorithms_parameters)

    return results

In [ ]:
import pandas as pd
# Die verschiedenen Algorithmen werden durchgegangen
algorithms_parameters = [] # Liste erstellen

algorithms_parameters.append( ( Gc_min_Neighbors, { "start" : 0 } ) )

algorithms_parameters.append( ( Gc_node_w, { "start" : 0} ) )

algorithms_parameters.append( ( Gc_rollout,{"start" : 0}))

algorithms_parameters.append( ( Gc_simplified_min_n,{"start": 0,"max_numbers_rollout":3}))

algorithms_parameters.append( ( Gc_simplified_node_w,{"start": 0,"max_numbers_rollout":3}))

algorithms_parameters.append( ( Graph_multi, {}))

algorithms_parameters.append( ( Graph_multi_node_w, {}))

algorithms_parameters.append( ( Graph_multi_rollout,{}))

algorithms_parameters.append( ( Gm_simplified_min_n,{"max_neighbors": 3}))

algorithms_parameters.append( ( Gm_simplified_node_w,{"max_neighbors": 3}))

results = run_algorithms_with_parameters_on_instances( verarbeite_ordner(ordner_klein), algorithms_parameters)

df_klein = pd.DataFrame(results)

results = run_algorithms_with_parameters_on_instances( verarbeite_ordner(ordner_groß), algorithms_parameters)
df_groß = pd.DataFrame(results)

df_results = pd.concat([df_klein,df_groß], ignore_index=True)

In [ ]:
df_results.to_latex("/content/drive/MyDrive/Ergebniss_Tabelle",escape=True)

### Algortihmen durchgehen (aufsummieren)

In [ ]:
def run_algorithm_with_parameters_sum(instance_name, function_algorithm, algorithm_parameters):
    # Die Ergebnisse der Algorithmen werden aufsummiert zum Gesamtüberblick
    result = 0

    # ausführen
    optimum = opt_rechner(instance_name)
    starttime = timer()
    permutation, distance = function_algorithm(instance_name, **algorithm_parameters)
    result += permutation
    time = timer() - starttime

    return result,optimum,time

In [ ]:
def run_algorithms_with_parameters_summarize(instance_name, algorithms_parameters):

    results = 0
    Optimum = 0
    Time = 0
    for algorithm, parameters in algorithms_parameters:

        Roboter,opt,time =  (run_algorithm_with_parameters_sum(instance_name, algorithm, parameters) )
        results += Roboter
        Optimum += opt
        Time += time

    return results,Optimum,time


def run_algorithms_with_parameters_on_instances_summarize (instance_names, algorithms_parameters):

    results = 0
    Optimum_gesamt = 0
    Time = 0
    for instance_name in instance_names:
        result,Optimum,time = run_algorithms_with_parameters_summarize( instance_name, algorithms_parameters)
        results += result
        Optimum_gesamt += Optimum
        Time += time
    return results, Optimum_gesamt,Time

Greedy_min_N = [( Gc_min_Neighbors, { "start" : 0 } )]
Greedy_node_w = [( Gc_node_w, { "start" : 0} )]
Multi_min_N = [( Graph_multi, {})]
Multi_node_w = [( Graph_multi_node_w, {})]
Rollout = [(Gc_rollout,{"start":0})]
simplified =  [(Gc_simplified_min_n,{"start": 0})]
simplified_node =  [( Gc_simplified_node_w,{"start": 0})]
multi_Rollout = [ ( Graph_multi_rollout,{})]
multi_simplified_min_n = [( Gm_simplified_min_n,{"max_neighbors": 3})]
multi_simplified_node_w = [( Gm_simplified_node_w,{"max_neighbors": 3})]

algo_parameter_liste = [Greedy_min_N,Greedy_node_w,Rollout,simplified,simplified_node,Multi_min_N,Multi_node_w,multi_Rollout,multi_simplified_min_n,multi_simplified_node_w]

for i,algo_para in enumerate (algo_parameter_liste,start = 1):
  Gesamt_klein,Optimum_klein,Zeit = run_algorithms_with_parameters_on_instances_summarize(verarbeite_ordner(ordner_klein),algo_para)
  Gesamt_groß,Optimum_groß,Zeit = run_algorithms_with_parameters_on_instances_summarize(verarbeite_ordner(ordner_groß),algo_para)

  print(f"Durchlauf für{algo_para}")
  print("Kleine Instanz")
  print(f"Gesamtmenge: {Gesamt_klein}, Optimum: {Optimum_klein},Zeit: {Zeit} Sekunden")
  print(" ")
  print("Große Instanz")
  print(f"Gesamtmenge: {Gesamt_groß}, Optimum: {Optimum_groß},Zeit: {Zeit} Sekunden")
  print(" ")

Durchlauf für[(<function Gc_min_Neighbors at 0x7de3653c8900>, {'start': 0})]
Kleine Instanz
Gesamtmenge: 817, Optimum: 471,Zeit: 0.027434180000000197 Sekunden
 
Große Instanz
Gesamtmenge: 2042, Optimum: 162,Zeit: 0.027434180000000197 Sekunden
 
Durchlauf für[(<function Gc_node_w at 0x7de3653c8b80>, {'start': 0})]
Kleine Instanz
Gesamtmenge: 1249, Optimum: 471,Zeit: 0.025682411999923715 Sekunden
 
Große Instanz
Gesamtmenge: 2499, Optimum: 162,Zeit: 0.025682411999923715 Sekunden
 
Durchlauf für[(<function Gc_rollout at 0x7de3653c8f40>, {'start': 0})]
Kleine Instanz
Gesamtmenge: 1118, Optimum: 471,Zeit: 5.042764604000013 Sekunden
 
Große Instanz
Gesamtmenge: 2138, Optimum: 162,Zeit: 5.042764604000013 Sekunden
 
Durchlauf für[(<function Gc_simplified_min_n at 0x7de3653c93a0>, {'start': 0})]
Kleine Instanz
Gesamtmenge: 937, Optimum: 471,Zeit: 1.2497383579998314 Sekunden
 
Große Instanz
Gesamtmenge: 2031, Optimum: 162,Zeit: 1.2497383579998314 Sekunden
 
Durchlauf für[(<function Gc_simplified